In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import time
import os

from bs4 import BeautifulSoup
import re
import pandas as pd

import requests
import os
import urllib
from urllib.parse import urlparse

import subprocess


In [2]:
IMPLICIT_DELAY = 60
EXPLICIT_DELAY = 30
TAKE_SCREENSHOT = False

In [3]:
#setup option for chrome profile
chrome_options = Options()

## NEVER USE THE FOLLOWING OPTION SINCE IT WILL MAKE SUPPOSEDLY DIFFERENT WINDOWS FOR DIFFERENT CLASS MIXED IN SOME WAY!!!!
# chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
## the following option is to solve the error if using the above cause a problem
# chrome_options.add_argument("--remote-debugging-port=9222") 

prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
chrome_options.add_experimental_option("prefs", prefs)
if TAKE_SCREENSHOT:
    chrome_options.add_argument('--headless')

#start web driver
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(IMPLICIT_DELAY)
wait = WebDriverWait(driver, EXPLICIT_DELAY)

In [4]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
      
def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, wait, xpath):

    try:
        wait.until(ec.presence_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element("xpath",xpath)

def sync_get_elements_by_xpath(driver, wait, xpath):

    try:
        wait.until(ec.presence_of_all_elements_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_elements("xpath",xpath)

In [5]:
root_address = 'https://learn.udacity.com/nanodegrees/nd880/parts/cd0569'
root_folder = './ai_for_trading_course_downloaded'

In [6]:
# start
driver.get(root_address)
os.makedirs(root_folder, exist_ok=True)

In [7]:
sync_get_element_by_xpath(driver, wait, "//div[text()='Sign in']").click()

email = sync_get_element_by_xpath(driver, wait, "//input[@id='email']")
password = sync_get_element_by_xpath(driver, wait, "//input[@id='revealable-password']")

email.send_keys("huohsien@gmail.com")
password.send_keys("Huo18941256")

sync_get_element_by_xpath(driver, wait, "//div[@data-testid='signin-form']//span[text()='Sign in']/..").click()


In [8]:
program_name = sync_get_element_by_xpath(driver, wait, "//h1[@class='chakra-heading css-10cnqhb']").get_attribute("innerHTML")
program_name

'AI for Trading'

In [9]:
course_tab_elms = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tab-panel css-1qbr3jw']")

tab_buttons = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tablist css-1cul31m']/button")
for tab_button in tab_buttons:
    print("Type: {}".format(tab_button.get_attribute("innerHTML")))


Type: Core
Type: Elective
Type: Career


In [10]:
course_content_blk1 = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tab-panels css-8atqhb']/div[1]//div[@class='css-pk1kta']")[0]
course_content_blk2 = sync_get_elements_by_xpath(driver, wait, "//div[@class='chakra-tabs__tab-panels css-8atqhb']/div[1]//div[@class='css-1vc4pt9']")[0] 

In [11]:
course_content_blk1.find_element("xpath", "./div/div/button/div/div[1]/div/div/h2").get_attribute("innerHTML")


'Quantitative Trading'

In [15]:
course_content_blk1.find_elements("xpath", "./div/div/div/div/a")

[<selenium.webdriver.remote.webelement.WebElement (session="8007fe56a3407c643760c496c57845fd", element="f.A733E8F9297CB18C0B70ECDDA0B5A9AF.d.837927CA243AFACA853D8F9A9E711B99.e.101")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8007fe56a3407c643760c496c57845fd", element="f.A733E8F9297CB18C0B70ECDDA0B5A9AF.d.837927CA243AFACA853D8F9A9E711B99.e.47")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8007fe56a3407c643760c496c57845fd", element="f.A733E8F9297CB18C0B70ECDDA0B5A9AF.d.837927CA243AFACA853D8F9A9E711B99.e.98")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8007fe56a3407c643760c496c57845fd", element="f.A733E8F9297CB18C0B70ECDDA0B5A9AF.d.837927CA243AFACA853D8F9A9E711B99.e.984")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8007fe56a3407c643760c496c57845fd", element="f.A733E8F9297CB18C0B70ECDDA0B5A9AF.d.837927CA243AFACA853D8F9A9E711B99.e.985")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8007fe56a3407c643760c4

In [12]:
e_header = sync_get_elements_by_xpath(driver, wait, "//div[@class='css-4nu87b']")
e_content = sync_get_elements_by_xpath(driver, wait, "//div[@class='css-dmdipe']")

header_height = int(e_header[0].get_attribute("scrollHeight"))
content_height = int(e_content[0].get_attribute("scrollHeight"))

full_page_height = content_height+header_height

full_page_width = int(e_header[0].get_attribute("scrollWidth"))

In [13]:
if TAKE_SCREENSHOT:

    driver.set_window_size(full_page_width,full_page_height)

    el = driver.find_element(By.TAG_NAME, "body")
    el.screenshot("main.png")
